In [ ]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [ ]:
gmsh.open("thermalStress.geo")

In [ ]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type="PlaneHeatConduction", bandwidth="none")

In [ ]:
K = FEM.heatConductionMatrix(problem)

In [ ]:
left = FEM.temperatureConstraint("left", T=100)
right = FEM.temperatureConstraint("right", T=100)
flux = FEM.heatFlux("left", qn=0)
#conv = FEM.heatConvection("outer", h=0.01, Tₐ=-10)

In [ ]:
q = FEM.loadVector(problem, [flux]);

In [ ]:
FEM.applyBoundaryConditions!(problem, K, q, [left, right])

In [ ]:
T = FEM.solveTemperature(K, q);

In [ ]:
Q = FEM.solveHeatFlux(problem, T)

In [ ]:
T1 = FEM.showDoFResults(problem, T, "T", name="T", visible=true)

In [ ]:
qq = FEM.showHeatFluxResults(problem, Q, "q", visible=false, smooth=false)

In [ ]:
FEM.plotOnPath(problem, "path", T1, 100, name="T", visible=false);

In [ ]:
problem2 = FEM.Problem([mat], type="PlaneStress", bandwidth="none")

In [ ]:
K2 = FEM.stiffnessMatrix(problem2)

In [ ]:
left2 = FEM.displacementConstraint("left", ux=0)
right2 = FEM.displacementConstraint("right", ux=0)
bottom2 = FEM.displacementConstraint("bottom", uy=0)
load = FEM.load("left", fx=0);

In [ ]:
f = FEM.loadVector(problem2, [load]);

In [ ]:
fT = FEM.thermalLoadVector(problem2, T);

In [ ]:
f2 = f + fT;

In [ ]:
FEM.applyBoundaryConditions!(problem2, K2, f2, [left2, right2, bottom2])
#FEM.applyBoundaryConditions!(problem2, K2, f2, [left2, bottom2])

In [ ]:
q2 = FEM.solveDisplacement(K2, f2);

In [ ]:
E = FEM.solveStrain(problem2, q2)

In [ ]:
S = FEM.solveStress(problem2, q2, T=T)

In [ ]:
u = FEM.showDoFResults(problem2, q2, "uvec", visible=false)
ux = FEM.showDoFResults(problem2, q2, "ux", visible=false)
uy = FEM.showDoFResults(problem2, q2, "uy", visible=false)

In [ ]:
ex = FEM.showStrainResults(problem2, E, "ex", visible=false)
ey = FEM.showStrainResults(problem2, E, "ey", visible=false)
exy = FEM.showStrainResults(problem2, E, "exy", visible=false)

In [ ]:
sx = FEM.showStressResults(problem2, S, "sx", visible=false)
sy = FEM.showStressResults(problem2, S, "sy", visible=false)
sxy = FEM.showStressResults(problem2, S, "sxy", visible=false)

In [ ]:
ΔT = 100
α = problem.material[1].α
ν = problem.material[1].ν
E = problem.material[1].E
l₀ = 100
Δl = l₀ * α * ΔT
ε₀ = α * ΔT
ε₁ = -Δl / l₀
ε₁ₖ = -ν * ε₁
σₓ = E * ε₁
p = E * α / 3 / (1 - 2ν) * ΔT
display("Δl = $Δl")
display("ε₀ = $ε₀")
display("ε₁ = $ε₁")
display("ε₁ₖ = $ε₁ₖ")
display("ε₁ₖ+ε₀ = $(ε₁ₖ+ε₀)")
display("σₓ = $σₓ")
display("p = $p")

In [ ]:
ε₁ = Δl / l₀

In [ ]:
FF = FEM.showDoFResults(problem2, fT, "uvec", name="fT", visible=false)

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()